In [1]:
import pandas as pd

In [2]:
applications_df = pd.read_csv("../data/application_record.csv")
credits_df = pd.read_csv("../data/credit_record.csv")

In [3]:
applications_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [4]:
credits_df.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [5]:
credits_df = credits_df[credits_df['MONTHS_BALANCE'] >-3]

In [6]:
applications_df.duplicated().any()

False

In [7]:
credits_df.duplicated().any()

False

In [8]:
applications_df = applications_df[applications_df['ID'].isin(credits_df['ID'])]
credits_df = credits_df[credits_df['ID'].isin(applications_df['ID'])]

In [9]:
applications_df = applications_df.drop(['CODE_GENDER', 'CNT_CHILDREN', 'NAME_FAMILY_STATUS', 'DAYS_BIRTH', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL'], axis=1)

In [10]:
def determine_approval(df):
    if(df['STATUS'] == 'C' or df['STATUS'] == 'X'): return 1
    else: return 0

In [11]:
credits_df['APPROVED'] = credits_df.apply(determine_approval, axis=1)

In [12]:
credits_df = credits_df.groupby('ID').agg(mean_approved=('APPROVED', 'mean')).reset_index()

In [13]:
credits_df['APPROVED'] = (credits_df['mean_approved'] > 0.5).astype(int)

In [15]:
credits_df = credits_df.drop('mean_approved', axis=1)

In [16]:
credits_df

,ID,APPROVED
0,5008804,1
1,5008805,1
2,5008806,1
3,5008808,0
4,5008810,1
...,...,...
25732,5150481,1
25733,5150483,1
25734,5150484,0
25735,5150485,0


In [17]:
merged_df = pd.merge(applications_df, credits_df, on='ID')

In [18]:
merged_df

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS,APPROVED
0,5008804,Y,Y,427500.0,Working,Higher education,Rented apartment,-4542,NaN,2.0,1
1,5008805,Y,Y,427500.0,Working,Higher education,Rented apartment,-4542,NaN,2.0,1
2,5008806,Y,Y,112500.0,Working,Secondary / secondary special,House / apartment,-1134,Security staff,2.0,1
3,5008808,N,Y,270000.0,Commercial associate,Secondary / secondary special,House / apartment,-3051,Sales staff,1.0,0
4,5008810,N,Y,270000.0,Commercial associate,Secondary / secondary special,House / apartment,-3051,Sales staff,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
25732,5149828,Y,Y,315000.0,Working,Secondary / secondary special,House / apartment,-2420,Managers,2.0,0
25733,5149834,N,Y,157500.0,Commercial associate,Higher education,House / apartment,-1325,Medicine staff,2.0,1
25734,5149838,N,Y,157500.0,Pensioner,Higher education,House / apartment,-1325,Medicine staff,2.0,1
25735,5150049,N,Y,283500.0,Working,Secondary / secondary special,House / apartment,-655,Sales staff,2.0,0


In [19]:
merged_df.isna().sum()

ID                        0
FLAG_OWN_CAR              0
FLAG_OWN_REALTY           0
AMT_INCOME_TOTAL          0
NAME_INCOME_TYPE          0
NAME_EDUCATION_TYPE       0
NAME_HOUSING_TYPE         0
DAYS_EMPLOYED             0
OCCUPATION_TYPE        7947
CNT_FAM_MEMBERS           0
APPROVED                  0
dtype: int64

In [20]:
merged_df = merged_df.dropna()

In [21]:
merged_df.isna().sum()

ID                     0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_HOUSING_TYPE      0
DAYS_EMPLOYED          0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
APPROVED               0
dtype: int64

In [22]:
merged_df[abs(merged_df['DAYS_EMPLOYED']) > 50000].count()

ID                     0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_HOUSING_TYPE      0
DAYS_EMPLOYED          0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
APPROVED               0
dtype: int64

In [23]:
merged_df['APPROVED'].value_counts()

APPROVED
1    11952
0     5838
Name: count, dtype: int64

In [24]:
merged_df = merged_df[merged_df['APPROVED'] != -1]

In [25]:
merged_df['APPROVED'].value_counts()

APPROVED
1    11952
0     5838
Name: count, dtype: int64

In [26]:
merged_df.to_csv("../data/merged_data.csv", index=False)